In [1]:
!pip3 install torch
!pip3 install torchvision
!pip3 install pandas

In [2]:


import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms, models
import pandas as pd
import os

In [3]:
# load the training-set
batch_size = 128

transform_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = datasets.ImageFolder(root='../cs-424-ass-1-wednesday-class/train', transform=transform_train)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

In [4]:
dev = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {dev}")

Using device: cuda


In [5]:
# Define ResNet-18 
class CustomResNet18(nn.Module):
    def __init__(self, num_classes):
        super(CustomResNet18, self).__init__()
        self.model = models.resnet18(weights=None)  # model from scratch
        self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)   
    def forward(self, x):
        return self.model(x)

num_classes = 10   
model = CustomResNet18(num_classes=num_classes).to(dev)

# Define the loss function and optimizer (INCREASE LOSS FUNCTION BC IT PLATEUED)
learning_rate = 0.02
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [6]:
def train_model():
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        for images, labels in train_loader:
            images, labels = images.to(dev), labels.to(dev)

            outputs = model(images)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(train_loader):.4f}")

    # save your model
    torch.save(model.state_dict(), 'resnet18_local.pth')

In [7]:
if __name__ == "__main__":
    num_epochs = 10
    train_model()

/common/home/users/n/naufala.2022/jupyterlab-venv-pytorch-240/lib/python3.11/site-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch [1/10], Loss: 3.2256
Epoch [2/10], Loss: 2.2132
Epoch [3/10], Loss: 1.9338
Epoch [4/10], Loss: 1.8194
Epoch [5/10], Loss: 1.7017
Epoch [6/10], Loss: 1.6173
Epoch [7/10], Loss: 1.5283
Epoch [8/10], Loss: 1.4522
Epoch [9/10], Loss: 1.3769
Epoch [10/10], Loss: 1.3521


In [8]:
# Model size (should be less than 26)

total_params = sum(p.numel() for p in model.parameters())
total_params/(1024*1024) 

10.663644790649414

In [9]:
# load the test-set
transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


test_dataset = datasets.ImageFolder(root='../cs-424-ass-1-wednesday-class/test', transform=transform_test)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [10]:
# test the model
def test_model():
    model.eval()
    predictions = []
    image_paths = [path for path, _ in test_dataset.imgs]   

    old_prefix = ".."
    new_prefix = "/kaggle/input"
    
    with torch.no_grad():
        for images, _ in test_loader:   
            images = images.to(dev)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            predictions.extend(predicted.cpu().numpy())

    adjusted_paths= [path.replace(old_prefix, new_prefix) for path in image_paths]

    # prediction.csv
    df = pd.DataFrame({
        'id': adjusted_paths,   
        'label': predictions
    })
    df.to_csv('Naufal Syaqil Bin Azmi.csv', index=False) # should be changed to Username.csv
    print("Results saved to Naufal Syaqil Bin Azmi.csv")

In [11]:
if __name__ == "__main__":
    test_model()

Results saved to Naufal Syaqil Bin Azmi.csv
